# Desafio final do curso

O projeto consiste em uma conciliação do valor total de vendas de recarga de celular por operadora e por loja. A Bemol recebe um arquivo de uma empresa financeira (detalhamento_m4u.xlsx) onde possui o valor consolidado de vendas e o objetivo é verificar se os dados do SAP (mcg3.csv) estão de acordo com o informado. Gerar um alerta caso os valores estejam divergentes.

Dica:
Os dados do SAP vêm separados por cada transição de venda realizada. A ideia é de agrupar esses dados para deixá-los no formato parecido com que recebemos da empresa externa e, assim, poder unificar as tabelas e fazer a comparação.

In [50]:
!ls

 desafio_final.ipynb	        LICENSE         relatorio_final.csv
 detalhamento_m4u.xlsx	        materiais.csv   relatorio_final.xlsx
'Estória do Usuário.xlsx'       mcg3.csv        vendas_1.xlsx
 fmad_aula_02.ipynb	        pedidos.txt     vendas_2.xlsx
 fmad_aula_03.ipynb	        Pipfile
 fmad_exercicio_aula_02.ipynb   README.md


In [51]:
import pandas as pd
import numpy as np

## Lendo os arquivos

In [52]:
!head -n 5 mcg3.csv

In [53]:
df_detalhamento = pd.read_excel('detalhamento_m4u.xlsx', skiprows=1)
df_detalhamento

,NomeFantasia,Escritório de vendas,UF,Operadora,Vlookup Bruto
0,BEMOL AVENIDA,103,AM,Claro,2651
1,BEMOL AVENIDA,103,AM,Oi,3309
2,BEMOL AVENIDA,103,AM,Tim,1550
3,BEMOL AVENIDA,103,AM,Vivo,4739
4,BEMOL BARROSO,107,AM,Claro,1495
...,...,...,...,...,...
121,BEMOL STUDIO 5,116,AM,Vivo,2144
122,BEMOL TORQUATO,500,AM,Claro,1838
123,BEMOL TORQUATO,500,AM,Oi,875
124,BEMOL TORQUATO,500,AM,Tim,560


In [54]:
df_sap = pd.read_csv('mcg3.csv', sep='|')
df_sap

,Escritório de vendas,Fornecedor,Material,Data,Val.líq.
0,101,VIVO,"131318 RECARGA VIRTUAL VIVO R$ 1,00",02.01.2020,20.0
1,101,VIVO,"131318 RECARGA VIRTUAL VIVO R$ 1,00",30.12.2019,10.0
2,101,VIVO,"131318 RECARGA VIRTUAL VIVO R$ 1,00",04.01.2020,20.0
3,101,VIVO,"131318 RECARGA VIRTUAL VIVO R$ 1,00",02.01.2020,10.0
4,101,VIVO,"131318 RECARGA VIRTUAL VIVO R$ 1,00",03.01.2020,25.0
...,...,...,...,...,...
5108,701,CLARO,"131317 RECARGA VIRTUAL CLARO R$ 1,00",30.12.2019,20.0
5109,701,CLARO,"131317 RECARGA VIRTUAL CLARO R$ 1,00",04.01.2020,20.0
5110,701,CLARO,"131317 RECARGA VIRTUAL CLARO R$ 1,00",05.01.2020,20.0
5111,701,CLARO,"131317 RECARGA VIRTUAL CLARO R$ 1,00",05.01.2020,45.0


## Tratamento de dados

### Renomeando Colunas

In [55]:
from unidecode import unidecode

In [56]:
# renomeando  df_sap
ind = [unidecode(col.lower().replace(" ", "_").replace("_de_", "_")) for col in df_detalhamento.columns]
ind[-1] = "val_bruto"
df_detalhamento.set_axis(ind, axis=1, inplace=True)

df_detalhamento.head()

,nomefantasia,escritorio_vendas,uf,operadora,val_bruto
0,BEMOL AVENIDA,103,AM,Claro,2651
1,BEMOL AVENIDA,103,AM,Oi,3309
2,BEMOL AVENIDA,103,AM,Tim,1550
3,BEMOL AVENIDA,103,AM,Vivo,4739
4,BEMOL BARROSO,107,AM,Claro,1495


In [57]:
# renomeando df_detalhamento
ind = [unidecode(col.lower().replace(" ", "_").replace("_de_", "_")) for col in df_sap.columns]
ind[-1] = "val_liq"
ind[1] = "operadora"
df_sap.set_axis(ind, axis=1, inplace=True)

df_sap.head()

,escritorio_vendas,operadora,material,data,val_liq
0,101,VIVO,"131318 RECARGA VIRTUAL VIVO R$ 1,00",02.01.2020,20.0
1,101,VIVO,"131318 RECARGA VIRTUAL VIVO R$ 1,00",30.12.2019,10.0
2,101,VIVO,"131318 RECARGA VIRTUAL VIVO R$ 1,00",04.01.2020,20.0
3,101,VIVO,"131318 RECARGA VIRTUAL VIVO R$ 1,00",02.01.2020,10.0
4,101,VIVO,"131318 RECARGA VIRTUAL VIVO R$ 1,00",03.01.2020,25.0


### Verificando os tipos das colunas

In [58]:
df_detalhamento.dtypes

nomefantasia         object
escritorio_vendas     int64
uf                   object
operadora            object
val_bruto             int64
dtype: object

In [59]:
df_sap.dtypes

escritorio_vendas      int64
operadora             object
material              object
data                  object
val_liq              float64
dtype: object

### Alterando tipos de dados de colunas

In [60]:
df_detalhamento.val_bruto = df_detalhamento.val_bruto.astype('float64')

### Alterando os nomes das operadoras

In [61]:
df_detalhamento.operadora = df_detalhamento.operadora.apply(lambda linha : linha.lower())

In [62]:
df_sap.operadora = df_sap.operadora.apply(lambda linha : linha.lower())

In [63]:
def change_name(linha):
    if linha == "tnl":
        return 'oi'
    else:
        return linha
    
df_sap.operadora = df_sap.operadora.apply(lambda linha : change_name(linha))

## Drop columns

#### df_detalhamento

In [64]:
df_detalhamento.drop(["uf", "nomefantasia"], axis=1, inplace=True)
df_detalhamento.head()

,escritorio_vendas,operadora,val_bruto
0,103,claro,2651.0
1,103,oi,3309.0
2,103,tim,1550.0
3,103,vivo,4739.0
4,107,claro,1495.0


#### df_sap

In [65]:
df_sap.drop(["material", "data"], axis=1, inplace=True)
df_sap.head()

,escritorio_vendas,operadora,val_liq
0,101,vivo,20.0
1,101,vivo,10.0
2,101,vivo,20.0
3,101,vivo,10.0
4,101,vivo,25.0


## Agrupamento

In [66]:
df_sap.loc[(df_sap["escritorio_vendas"] == 101) & (df_sap["operadora"] == "vivo")]["val_liq"].sum()

3142.0

In [67]:
df_sap.groupby(["escritorio_vendas", "operadora"])["val_liq"].sum()

escritorio_vendas  operadora
101                claro        1527.0
                   oi           1730.0
                   tim           975.0
                   vivo         3142.0
103                claro        2652.3
                                 ...  
618                vivo          561.0
701                claro         140.0
                   oi             20.0
                   tim           135.0
                   vivo          304.0
Name: val_liq, Length: 163, dtype: float64

In [68]:
df_sap = df_sap.groupby(["escritorio_vendas", "operadora"]).agg({'val_liq':'sum'}).reset_index()
df_sap

,escritorio_vendas,operadora,val_liq
0,101,claro,1527.0
1,101,oi,1730.0
2,101,tim,975.0
3,101,vivo,3142.0
4,103,claro,2652.3
...,...,...,...
158,618,vivo,561.0
159,701,claro,140.0
160,701,oi,20.0
161,701,tim,135.0


## Merge

In [69]:
df_sap.shape, df_detalhamento.shape

((163, 3), (126, 3))

In [70]:
df = pd.merge(df_sap, df_detalhamento, on=["escritorio_vendas", "operadora"], how="left")
df.head()

,escritorio_vendas,operadora,val_liq,val_bruto
0,101,claro,1527.0,1527.0
1,101,oi,1730.0,1730.0
2,101,tim,975.0,975.0
3,101,vivo,3142.0,3142.0
4,103,claro,2652.3,2651.0


In [71]:
df_detalhamento.loc[df_detalhamento["escritorio_vendas"] == 618]

,escritorio_vendas,operadora,val_bruto
67,618,claro,90.0
68,618,oi,130.0
69,618,tim,85.0
70,618,vivo,380.0


In [72]:
df_sap.loc[df_sap["escritorio_vendas"] == 618]

,escritorio_vendas,operadora,val_liq
155,618,claro,120.0
156,618,oi,255.0
157,618,tim,85.0
158,618,vivo,561.0


In [73]:
df.loc[df["escritorio_vendas"] == 118]

,escritorio_vendas,operadora,val_liq,val_bruto
40,118,claro,1265.0,1265.0
41,118,oi,1045.0,1045.0
42,118,tim,365.0,365.0
43,118,vivo,2147.0,2147.0


## Preenchedo os valores faltantes de val_bruto

In [74]:
df.val_bruto.fillna(0, inplace=True)
df.sample(50)

,escritorio_vendas,operadora,val_liq,val_bruto
11,105,vivo,1943.0,1943.0
111,607,claro,2498.0,2498.0
92,602,oi,1369.0,1369.0
129,611,tim,10.0,10.0
17,107,oi,1925.0,1925.0
74,500,tim,560.0,560.0
130,611,vivo,1698.0,1658.0
102,604,vivo,4474.0,4474.0
29,115,oi,2620.0,2635.0
1,101,oi,1730.0,1730.0


## Aplicando regra de negócio

In [75]:
def verify_status(val_liq, val_bruto):
    if val_liq == val_bruto:
        return "ok"
    else:
        return "Alerta"

In [76]:
df.val_liq

0      1527.0
1      1730.0
2       975.0
3      3142.0
4      2652.3
        ...  
158     561.0
159     140.0
160      20.0
161     135.0
162     304.0
Name: val_liq, Length: 163, dtype: float64

In [78]:
df["status"] = df.apply(lambda row : verify_status(row["val_liq"], row["val_bruto"]), axis=1)

In [80]:
df.head(30)

,escritorio_vendas,operadora,val_liq,val_bruto,status
0,101,claro,1527.00,1527.0,ok
1,101,oi,1730.00,1730.0,ok
2,101,tim,975.00,975.0,ok
3,101,vivo,3142.00,3142.0,ok
4,103,claro,2652.30,2651.0,Alerta
5,103,oi,3309.00,3309.0,ok
6,103,tim,1550.00,1550.0,ok
7,103,vivo,4739.00,4739.0,ok
8,105,claro,720.00,720.0,ok
9,105,oi,2030.00,2030.0,ok


## Exportando para planilha excel

In [83]:
writer = pd.ExcelWriter("desafio_final_relatorio.xlsx")

df_sap.to_excel(writer, sheet_name="sap", index=False)
df_detalhamento.to_excel(writer, sheet_name="detalhamento", index=False)
df.to_excel(writer, sheet_name="relatorio", index=False)

writer.save()